# 隐语SecretFlow金融风控全链路能力展示

> This tutorial is only available in Chinese.

> Last updated: Oct 7, 2023
>
> 请使用v0.8.3或以上版本的隐语进行实验。
>
> 以下代码仅作为示例，请勿在生产环境直接使用。

本次实验将会展示如何使用隐语进行在风控领域常用的Logistic Regeression模型和XGB模型的模型研发工作。

隐语接下来将会开放模型部署和在线/离线模型预测功能，敬请期待。

## 实验目标

在本次实验中，我们将会利用一个开源数据集训练一个金融风控场景常用的线性回归和XGB模型。在此过程中将包含以下步骤：

- 样本对齐
- 特征预处理
- 数据分析
- 模型训练
- 模型预测
- 模型评估

请依次执行所有步骤确保实验可以顺利完成。

## 实验前置工作

### 初始化隐语框架

在本次实验中，我们将会包含两个节点：**alice** 和 **bob** . 在真实业务场景，他们将会代表两个不同实体，他们之间的原始数据不被允许直接相互传输，但是他们的原始数据将会被一起用以研发一个模型。

在下面的代码中，我们建立了一个 **SecretFlow Cluster**, 基于 **alice** 和 **bob** 两个节点，我们还创建了三个device：

- alice: PYU device, 负责在alice侧的本地计算，计算输入、计算过程和计算结果仅alice可见
- bob: PYU device, 负责在bob侧的本地计算，计算输入、计算过程和计算结果仅bob可见
- spu: SPU device, 负责alice和bob之间的密态计算，计算输入和计算结果为密态，由alice和bob各掌握一个分片，计算过程为MPC计算，由alice和bob各自的SPU Runtime一起执行。

>  如果你尚未理解以上的一些概念，比如SPU设备，请参考这篇[文档](../developer/design/architecture.md).


In [79]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

sf.shutdown()

sf.init(['alice', 'bob'], address='local')
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

The version of SecretFlow: 1.8.0b0


/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-22 21:02:21,798	INFO worker.py:1724 -- Started a local Ray instance.


In [ ]:
"""
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

sf.shutdown()
cluster_config = {
    'parties': {
        'alice': {
            # replace with alice's real address.
            'address': 'ecm-01:16307',
            'listen_addr': '0.0.0.0:16307'
        },
        'bob': {
            # replace with bob's real address.
            'address': 'ecm-02:16307',
            'listen_addr': '0.0.0.0:16307'
        },
    },
    'self_party': 'alice'
}

# sf.init(address='211.87.232.46:6207', cluster_config=cluster_config,tls_config=tls_config)
sf.init(address='ecm-01:6379', cluster_config=cluster_config)
alice, bob = sf.PYU('alice'), sf.PYU('bob')

cluster_def={'nodes': [{'party': 'alice', 'address': 'ecm-01:11666'},
  {'party': 'bob', 'address': 'ecm-02:11666'}],
 'runtime_config': {'protocol': 2, 'field': 3}}

spu = sf.SPU(cluster_def)

"""

在上面的log中，你应该发现，在**spu**的创建过程中，alice和bob两边都各有一个 **SPURuntime** 被建立并互相创建连接。

### 数据集

本次实验我们采用的原始数据是来自UCI的[Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing). 这个数据集汇集了一家葡萄牙银行机构电话营销的结果。

我们添加了**uid**这一列用于接下来隐私求交的实验。

我们首先看一下数据集所包含的信息。


In [80]:
import pandas as pd

# secretflow.utils.simulation.datasets contains mirrors of some popular open dataset.
from secretflow.utils.simulation.datasets import dataset

df = pd.read_csv(dataset('bank_marketing_full'), sep=';')
df['uid'] = df.index + 1

df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,3
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,4
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,45207
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,45208
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,45209
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,45210


该数据集包含了45211个样本，每一个样本代表了一个目标客户。

每个样本包含16个feature，我们这里简单描述一下这个数据集所有的feature。


| feature | 描述 | 取值 |
| :-----| :---- | :---- |
| uid | 客户编码 | 数字 |
| age | 年龄 | 数字 |
| job | 工作类型 |  'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown' |
| marital | 婚姻状况 | 'divorced','married','single','unknown' |
| education | 教育状况 | 'tertiary', 'secondary', 'unknown', 'primary' |
| default | 是否有不良信用记录 | 'no','yes','unknown' |
| housing | 是否有房贷 |  'no','yes','unknown' |
| loan | 是否有个人贷款 | 'no','yes','unknown' |
| contact | 联系方式 | 'cellular','telephone' |
| month | 上次联系月份 | 'jan', 'feb', 'mar', ..., 'nov', 'dec' |
| day | 上次联系月日 |数字|
| duration | 上次沟通时间 | 数字 |
| campaign | 本次活动已经沟通的次数 | 数字 |
| pdays | 距离上次沟通经过的天数 | 数字 |
| previous | 在本次活动之前已经沟通的次数 | 数字 |
| poutcome | 之前活动的结果 | 'unknown', 'failure', 'other', 'success' | 




每个样本的label - y表示对于目标客户的营销结果（是否签订了定额存款合同），取值是'yes','no'。

我们假定以上16个feature由两个机构分别掌握，具体如下。

- alice: age, job, marital, education, default, balance, housing, loan
- bob: contact, day, month, duration, campaign, pdays, previous, poutcome, y


在真实业务场景中, alice和bob所掌握的数据可能是没有对齐的，为了模拟这种情况，我们将数据集shuffle之后，再随机各取90%来模拟这个状况。

In [81]:
import numpy as np

df_alice = df.iloc[:, np.r_[0:8, -1]].sample(frac=0.9)

df_alice

,age,job,marital,education,default,balance,housing,loan,uid
22020,35,entrepreneur,divorced,secondary,no,80,no,no,22021
21764,53,technician,married,secondary,no,428,no,no,21765
43337,43,management,divorced,tertiary,no,2367,no,no,43338
15172,30,management,single,tertiary,no,1289,no,no,15173
44763,28,management,single,tertiary,no,187,no,no,44764
...,...,...,...,...,...,...,...,...,...
27424,34,unemployed,married,secondary,no,6438,yes,no,27425
44249,35,admin.,single,secondary,no,321,no,no,44250
19643,44,technician,married,secondary,no,1371,no,no,19644
2030,48,blue-collar,married,primary,no,418,yes,no,2031


In [82]:
df_bob = df.iloc[:, 8:].sample(frac=0.9)

df_bob

,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
24481,telephone,17,nov,14,1,-1,0,unknown,no,24482
44062,unknown,2,jul,48,1,422,1,success,no,44063
36461,cellular,11,may,63,2,370,2,failure,no,36462
32029,cellular,14,apr,145,1,-1,0,unknown,no,32030
16704,cellular,24,jul,250,1,-1,0,unknown,no,16705
...,...,...,...,...,...,...,...,...,...,...
43351,cellular,25,mar,151,1,184,2,failure,no,43352
26017,cellular,19,nov,363,2,-1,0,unknown,no,26018
29597,cellular,3,feb,65,2,-1,0,unknown,no,29598
32638,cellular,17,apr,28,1,345,1,failure,no,32639


我们这里将df_alice和df_bob保存为文件，作为alice和bob两方的原始输入。

至此，我们完成了所有实验准备工作。

In [83]:
import tempfile

_, alice_path = tempfile.mkstemp()
_, bob_path = tempfile.mkstemp()
df_alice.reset_index(drop=True).to_csv(alice_path, index=False)
df_bob.reset_index(drop=True).to_csv(bob_path, index=False)

## 样本对齐（隐私求交）

显然，第一步我们需要将两边的数据对齐。
隐私求交（[Private Set Intersection](https://en.wikipedia.org/wiki/Private_set_intersection))是一种密码学方法，可以获取两个集合的交集，而不泄露任何其他信息。
在隐语中，SPU设备支持三种隐私求交算法:

- [ECDH](https://ieeexplore.ieee.org/document/6234849/)：半诚实模型, 基于公钥密码学，原本适用于小数据集，但是隐语优化后已经能支持10亿量级的数据。
- [KKRT](https://eprint.iacr.org/2016/799.pdf)：半诚实模型, 基于布谷鸟哈希（Cuckoo Hashing）以及高效不经意传输扩展（OT Extension），适用于大数据集（比如千万数据集）。
- [BC22PCG](https://eprint.iacr.org/2022/334)：半诚实模型, 基于随机相关函数生成器，适用于大数据集。

由于我们这里的数据集较小，我们这里采用的是ECDH方法。


### 方式一：将隐私求交结果保存至文件

在一些应用场景场景中，alice和bob可能在隐私求交之后将结果直接保存至文件中，之后再进行后续操作。这个时候，请调用**psi_csv**接口。

在以下代码中，我们分别制定了两边需要求交的key以及输入和输出路径。

我们需要指定双方的输入文件和输出文件路径。对于ECDH来说，由于双方的地位是平等的，receiver并没有实际含义，你可以任意指定。我们需要设定正确的protocol。sort设为true之后，join的结果将会被排序。

> 请阅读 psi_csv 的文档。

In [11]:
_, alice_psi_path = tempfile.mkstemp()
_, bob_psi_path = tempfile.mkstemp()

spu.psi_csv(
    key="uid",
    input_path={alice: alice_path, bob: bob_path},
    output_path={alice: alice_psi_path, bob: bob_psi_path},
    receiver="alice",
    protocol="ECDH_PSI_2PC",
    sort=True,
)

[{'party': 'alice', 'original_count': 40690, 'intersection_count': 36621},
 {'party': 'bob', 'original_count': 40690, 'intersection_count': 36621}]

### 方式二：将求交结果保存至VDataFrame

VDataFrame是隐语中保存垂直切分数据的数据结构，在接下来的任务中，我们将会不断使用VDataFrame的数据结构。

由于在本次实验中，经过隐私求交之后，我们还有后续操作，所以我们在这里使用 **data.vertical.read_csv** 来将原始数据隐私求交之后的结果直接转化为VDataFrame。

> 请阅读data.vertical.read_csv的文档。很多参数和psi_csv是一致的，这里不再赘述。

In [84]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(
    {alice: alice_path, bob: bob_path},
    spu=spu,
    keys="uid",
    drop_keys="uid",
    psi_protocl="ECDH_PSI_2PC",
)
vdf.columns

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

In [16]:
len(vdf.columns)

17

### 更多

我们在这里展示的是两方单键的隐私求交，隐语也支持三方和多键的隐私求交技术，想要了解更多信息，你可以：

- 阅读这篇[文档](https://www.secretflow.org.cn/docs/spu/en/development/psi.html)了解隐语SPU的隐私求交能力。
- 阅读该[教程](./PSI_On_SPU.ipynb)了解使用的例子。

## 特征预处理

一般情况下，我们都需要对用于建模的数据进行预处理，合理的预处理对模型训练效果非常关键。

在开始特征预处理之前，我们先使用 **stats.table_statistics.table_statistics** 来查看一下特征总体情况，我们会在后面专门讨论全表统计模块。

In [85]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

,datatype,total_count,count(non-NA count),count_na(NA count),na_ratio,min,max,mean,var(variance),std(standard deviation),...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
age,int64,36593,36593,0,0.0,18.0,95.0,40.902905,1.128156e+02,10.621471,...,1.785860e+03,8.310418e+04,4.109583e+06,1.128126e+02,8.285922e+02,4.248222e+04,1496760.0,6.534998e+07,3.041031e+09,1.503820e+11
job,object,36593,36593,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marital,object,36593,36593,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education,object,36593,36593,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
default,object,36593,36593,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
balance,int64,36593,36593,0,0.0,-8019.0,102127.0,1356.367338,9.285826e+06,3047.265322,...,1.112530e+07,2.819807e+11,3.984372e+15,9.285572e+06,2.417014e+11,1.315337e+16,49633550.0,4.071083e+11,1.031852e+16,-1.773821e+18
housing,object,36593,36593,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan,object,36593,36593,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
contact,object,36593,36593,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
day,int64,36593,36593,0,0.0,1.0,31.0,15.805209,6.921922e+01,8.319809,...,3.190219e+02,7.282557e+03,1.787461e+05,6.921732e+01,5.236028e+01,9.288616e+03,578360.0,1.167397e+07,2.664906e+08,6.540855e+09


In [86]:
pd.reset_option('display.max_rows')

在接下来，我们将会展示隐语以下特征预处理能力：

- 值替换
- 缺失值填充
- WOE分组/分箱转换
- one-hot编码
- 标准化

### 值替换

我们先对以下特征做值替换：

| feature | 描述 | 取值和值替换规则 |
| :-----| :---- | :---- |
| education | 教育状况 | 'tertiary' -> 3, 'secondary' -> 2, 'unknown' -> 0, 'primary' -> 1 |
| default | 是否有不良信用记录 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| housing | 是否有房贷 |  'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| loan | 是否有个人贷款 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| month | 上次联系月份 | 'jan' -> 1, 'feb' -> 2, 'mar' -> 3, ..., 'nov' -> 11, 'dec' ->12 |
| y | label | 'yes' -> 1,'no' -> 0 |


替换完之后，我们使用 **sf.reveal** 来查看效果，请注意在生产中，**sf.reveal** 将会直接泄露数据，需要严格限制和进行审计。

> 在生产中，请严格限制**sf.reveal**的使用。

In [87]:
vdf['education'] = vdf['education'].replace(
    {'tertiary': 3, 'secondary': 2, 'primary': 1, 'unknown': np.NaN}
)

vdf['default'] = vdf['default'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['housing'] = vdf['housing'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['loan'] = vdf['loan'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['month'] = vdf['month'].replace(
    {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr': 4,
        'may': 5,
        'jun': 6,
        'jul': 7,
        'aug': 8,
        'sep': 9,
        'oct': 10,
        'nov': 11,
        'dec': 12,
    }
)

vdf['y'] = vdf['y'].replace(
    {
        'no': 0,
        'yes': 1,
    }
)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

       age           job  marital  education  default  balance  housing  loan
0       58    management  married        3.0        0     2143        1     0
1       46    management  married        3.0        0      229        1     0
2       33    technician   single        2.0        0       56        0     0
3       42    technician  married        2.0        0     8036        0     0
4       38        admin.  married        1.0        0     1487        0     0
...    ...           ...      ...        ...      ...      ...      ...   ...
36588   49    technician  married        2.0        0      378        0     0
36589   49     housemaid  married        1.0        0     3376        0     0
36590   52  entrepreneur  married        3.0        0     1115        1     0
36591   40   blue-collar  married        1.0        0       48        0     0
36592   46      services  married        3.0        0      474        0     0

[36593 rows x 8 columns]
       contact  day  month  duration  

#### 安全性讨论

值替换操作由数据所有者的PYU Device执行，不会泄露数据。

### 缺失值填充

接下来我们对缺失值进行填充。我们在这里均填充了众数，其他可选的策略还包括平均数、中位数等。

其他可能的处理方法包括删除缺省的行, 或者可以使用数据完整的行作为训练集，以此来预测缺失值。

替换完之后，我们使用 **sf.reveal** 来查看效果。

In [88]:
vdf["education"] = vdf["education"].fillna(vdf["education"].mode())
vdf["default"] = vdf["default"].fillna(vdf["default"].mode())
vdf["housing"] = vdf["housing"].fillna(vdf["housing"].mode())
vdf["loan"] = vdf["loan"].fillna(vdf["loan"].mode())

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

       age           job  marital  education  default  balance  housing  loan
0       58    management  married        3.0        0     2143        1     0
1       46    management  married        3.0        0      229        1     0
2       33    technician   single        2.0        0       56        0     0
3       42    technician  married        2.0        0     8036        0     0
4       38        admin.  married        1.0        0     1487        0     0
...    ...           ...      ...        ...      ...      ...      ...   ...
36588   49    technician  married        2.0        0      378        0     0
36589   49     housemaid  married        1.0        0     3376        0     0
36590   52  entrepreneur  married        3.0        0     1115        1     0
36591   40   blue-collar  married        1.0        0       48        0     0
36592   46      services  married        3.0        0      474        0     0

[36593 rows x 8 columns]
       contact  day  month  duration  

#### 安全性讨论

所填充的缺失值由属于数据所有者的PYU Device执行，并在接下来的缺失值操作中由数据所有者的PYU Device使用，不会泄露数据。

### woe分箱

woe分箱用于将连续值替换为离散值。

将连续型特征离散化的一个好处是可以有效地克服数据中隐藏的缺陷： 使模型结果更加稳定。例如，数据中的极端值是影响模型效果的一个重要因素。极端值导致模型参数过高或过低，或导致模型被虚假现象"迷惑"，把原来不存在的关系作为重要模式来学习。而离散化可以有效地减弱极端值和异常值的影响。

变量duration的75%分位数远小于最大值，而且该变量的标准差相对也比较大。因此需要对变量duration进行离散化。

In [89]:
from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning
from secretflow.preprocessing.binning.vert_bin_substitution import VertBinSubstitution

binning = VertWoeBinning(spu)
bin_rules = binning.binning(
    vdf,
    binning_method="chimerge",
    bin_num=4,
    bin_names={alice: [], bob: ["duration"]},
    label_name="y",
)

woe_sub = VertBinSubstitution()
vdf, _ = woe_sub.substitution(vdf, bin_rules)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorVertWoeBinningPyuWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorVertWoeBinningPyuWorker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


       age           job  marital  education  default  balance  housing  loan
0       58    management  married        3.0        0     2143        1     0
1       46    management  married        3.0        0      229        1     0
2       33    technician   single        2.0        0       56        0     0
3       42    technician  married        2.0        0     8036        0     0
4       38        admin.  married        1.0        0     1487        0     0
...    ...           ...      ...        ...      ...      ...      ...   ...
36588   49    technician  married        2.0        0      378        0     0
36589   49     housemaid  married        1.0        0     3376        0     0
36590   52  entrepreneur  married        3.0        0     1115        1     0
36591   40   blue-collar  married        1.0        0       48        0     0
36592   46      services  married        3.0        0      474        0     0

[36593 rows x 8 columns]
       contact  day  month  duration  

#### 安全性讨论

woe分桶需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### One Hot编码

one-hot编码适用于将类型编码转化为数值编码。 对于job、marital等特征我们需要one-hot编码。

In [39]:
from secretflow.preprocessing.encoder import OneHotEncoder

encoder = OneHotEncoder()
# for vif and correlation only
vdf_hat = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

tranformed_df = encoder.fit_transform(vdf['job'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['marital'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['contact'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['month'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['day'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['poutcome'])
vdf[tranformed_df.columns] = tranformed_df

vdf = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

print(type(sf.reveal(vdf.partitions[alice].data)))
print(sf.reveal(vdf.partitions[bob].data))

       age  education  default  balance  housing  loan  job_admin.  \
0       33        2.0        0       56        0     0         0.0   
1       38        2.0        0      272        1     1         1.0   
2       40        2.0        0      -63        0     0         0.0   
3       58        1.0        0      593        0     1         0.0   
4       25        2.0        0       66        1     0         1.0   
...    ...        ...      ...      ...      ...   ...         ...   
36616   36        3.0        0     4844        0     0         0.0   
36617   49        2.0        0      378        0     0         0.0   
36618   49        1.0        0     3376        0     0         0.0   
36619   40        1.0        0       48        0     0         0.0   
36620   46        3.0        0      474        0     0         0.0   

       job_blue-collar  job_entrepreneur  job_housemaid  ...  job_retired  \
0                  0.0               0.0            0.0  ...          0.0   
1    

#### 安全性讨论

one-hot编码操作由数据所有者的PYU Device执行，不会泄露数据。

### 标准化 
特征之间数值差距太大会使得模型收敛困难，我们一般先对数值进行标准化。

In [44]:
from secretflow.preprocessing import StandardScaler

X = vdf.drop(columns=['y'])
y = vdf['y']
scaler = StandardScaler()
X = scaler.fit_transform(X)
vdf[X.columns] = X
print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

            age  education   default   balance   housing      loan  \
0     -0.750042  -0.221796 -0.133369 -0.431496 -1.118776 -0.435400   
1     -0.279204  -0.221796 -0.133369 -0.360123  0.893834  2.296741   
2     -0.090869  -0.221796 -0.133369 -0.470817 -1.118776 -0.435400   
3      1.604150  -1.755189 -0.133369 -0.254055 -1.118776  2.296741   
4     -1.503384  -0.221796 -0.133369 -0.428192  0.893834 -0.435400   
...         ...        ...       ...       ...       ...       ...   
36616 -0.467539   1.311597 -0.133369  1.150602 -1.118776 -0.435400   
36617  0.756641  -0.221796 -0.133369 -0.325098 -1.118776 -0.435400   
36618  0.756641  -1.755189 -0.133369  0.665531 -1.118776 -0.435400   
36619 -0.090869  -1.755189 -0.133369 -0.434140 -1.118776 -0.435400   
36620  0.474138   1.311597 -0.133369 -0.293376 -1.118776 -0.435400   

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  ...  \
0       -0.361375        -0.520820          -0.18619      -0.166861  ...   
1      

#### 安全性讨论

标准化操作由数据所有者的PYU Device执行，不会泄露数据。

### 更多

隐语还支持其他更多的特征预处理能力，请参考这篇[文档](./data_preprocessing_with_data_frame.ipynb).

至此，我们已经完成了所有特征预处理工作。

> 本文主要目的是为了展示隐语的预处理能力，本文对于数据预处理方法的使用可能是有争议的，敬请谅解。

## 数据分析

在建模之前，我们有必要分析一下我们所使用的数据，以便确认是否需要重复特征预处理的过程。

下面我们将会展示隐语以下数据分析能力:

- 全表统计
- 相关系数矩阵
- VIF指标计算


### 全表统计

我们提供了类似于 **pd.DataFrame.describe** 来展示所有特征的基本统计信息。

> 在特征预处理的过程中，你可以不断调用全表统计来关注预处理效果。

In [45]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

,datatype,total_count,count(non-NA count),count_na(NA count),na_ratio,min,max,mean,var(variance),std(standard deviation),...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
age,float64,36621,36621,0,0.0,-2.162558,5.088355,2.219658e-16,1.000027,1.000014,...,1.000000,0.681324,3.309332,1.000000,0.681324,3.309332,8.128609e-12,36621.0,2.495078e+04,1.211910e+05
education,float64,36621,36621,0,0.0,-1.755189,1.311597,-3.104417e-16,1.000027,1.000014,...,1.000000,-0.154791,2.306711,1.000000,-0.154791,2.306711,-1.136868e-11,36621.0,-5.668599e+03,8.447405e+04
default,float64,36621,36621,0,0.0,-0.133369,7.498021,-2.793975e-17,1.000027,1.000014,...,1.000000,7.364652,55.238100,1.000000,7.364652,55.238100,-1.023182e-12,36621.0,2.697009e+05,2.022874e+06
balance,float64,36621,36621,0,0.0,-3.099717,33.295811,1.552208e-17,1.000027,1.000014,...,1.000000,8.328070,145.743259,1.000000,8.328070,145.743259,5.684342e-13,36621.0,3.049823e+05,5.337264e+06
housing,float64,36621,36621,0,0.0,-1.118776,0.893834,1.241767e-17,1.000027,1.000014,...,1.000000,-0.224942,1.050599,1.000000,-0.224942,1.050599,4.547474e-13,36621.0,-8.237613e+03,3.847399e+04
loan,float64,36621,36621,0,0.0,-0.435400,2.296741,-5.161092e-17,1.000027,1.000014,...,1.000000,1.861341,4.464590,1.000000,1.861341,4.464590,-1.890044e-12,36621.0,6.816417e+04,1.634977e+05
job_admin.,float64,36621,36621,0,0.0,-0.361375,2.767209,-2.444728e-17,1.000027,1.000014,...,1.000000,2.405834,6.788039,1.000000,2.405834,6.788039,-8.952838e-13,36621.0,8.810406e+04,2.485848e+05
job_blue-collar,float64,36621,36621,0,0.0,-0.520820,1.920049,-1.862650e-17,1.000027,1.000014,...,1.000000,1.399229,2.957842,1.000000,1.399229,2.957842,-6.821210e-13,36621.0,5.124116e+04,1.083191e+05
job_entrepreneur,float64,36621,36621,0,0.0,-0.186190,5.370844,-5.122287e-17,1.000027,1.000014,...,1.000000,5.184654,27.880633,1.000000,5.184654,27.880633,-1.875833e-12,36621.0,1.898672e+05,1.021017e+06
job_housemaid,float64,36621,36621,0,0.0,-0.166861,5.993023,1.707429e-17,1.000027,1.000014,...,1.000000,5.826163,34.944173,1.000000,5.826163,34.944173,6.252776e-13,36621.0,2.133599e+05,1.279691e+06


In [46]:
pd.reset_option('display.max_rows')

#### 安全性讨论

请注意，全表统计会暴露数据整体统计结果，其背后实际上蕴含了**sf.reveal**,请谨慎使用。

### 相关系数矩阵

我们接下来计算特征和特征之间，特征和标签之间的相关系数矩阵。

> 计算相关系数矩阵时，one-hot编码各列无需参与计算。

In [47]:
from secretflow.stats.ss_pearsonr_v import PearsonR

pearson_r_calculator = PearsonR(spu)
corr_matrix = pearson_r_calculator.pearsonr(vdf_hat)

import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
corr_matrix

array([[1.000, -0.164, -0.020, 0.096, -0.185, -0.015, -0.057, -0.043,
        0.020, 0.086, -0.024, 0.446, -0.006, -0.067, -0.196, -0.068,
        0.003, 0.051, -0.006, 0.004, -0.028, -0.003, 0.027],
       [-0.164, 1.000, -0.014, 0.072, -0.077, -0.027, -0.042, -0.412,
        0.052, -0.126, 0.513, -0.116, 0.086, -0.088, 0.006, 0.068,
        -0.032, -0.024, -0.007, 0.007, 0.003, 0.025, 0.067],
       [-0.020, -0.014, 1.000, -0.066, -0.004, 0.076, -0.008, 0.006,
        0.030, 0.003, -0.005, -0.010, 0.003, 0.004, -0.016, -0.004,
        0.008, -0.006, -0.005, 0.013, -0.030, -0.019, -0.021],
       [0.096, 0.072, -0.066, 1.000, -0.072, -0.084, -0.027, -0.046,
        0.008, 0.002, 0.072, 0.043, 0.018, -0.042, -0.002, -0.016, 0.008,
        0.011, 0.016, -0.015, 0.004, 0.016, 0.051],
       [-0.185, -0.077, -0.004, -0.072, 1.000, 0.043, 0.042, 0.181,
        0.008, -0.079, -0.067, -0.156, -0.024, 0.077, -0.082, -0.017,
        -0.048, -0.076, -0.005, -0.026, 0.126, 0.037, -0.137],
      

#### 安全性讨论

相关系数矩阵的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### VIF指标计算

隐语还支持VIF的计算来进行多重共线性检验。

> 计算VIF指标时，one-hot编码各列无需参与计算。

In [48]:
from secretflow.stats.ss_vif_v import VIF

vif_calculator = VIF(spu)
vif_results = vif_calculator.vif(vdf_hat)
print(vdf_hat.columns)
print(vif_results)

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


['age', 'education', 'default', 'balance', 'housing', 'loan', 'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'duration', 'campaign', 'pdays', 'previous', 'y']
[1.409 1.705 1.013 1.033 1.149 1.026 5413.513 8893.329 1716.573 1397.236
 8806.741 2504.609 1800.107 4399.248 1055.511 7398.044 1499.605 341.647
 1.168 1.054 1.277 1.241 1.183]


#### 安全性讨论

VIF指标的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

## 模型训练

接下来，我们将会分别训练一个逻辑回归模型和一个XGB模型。


### 随机分割

在训练之前，我们需要将数据分割为训练集和验证集。

其中train_x和train_y为训练集的特征和标签。test_x和test_y为训练集的特征和标签。


In [49]:
from secretflow.data.split import train_test_split

random_state = 1234

train_vdf, test_vdf = train_test_split(vdf, train_size=0.8, random_state=random_state)

train_x = train_vdf.drop(columns=['y'])
train_y = train_vdf['y']

test_x = test_vdf.drop(columns=['y'])
test_y = test_vdf['y']

#### 安全性讨论

随机分割时，每一方会共享随机数种子，并由每一方数据的owner分别执行各自的数据分割并且确保最终分割结果仍然是对齐的。

### PSI（人群稳定性分析）

样本稳定指数是衡量样本变化所产生的偏移量的一种重要指标，通常用来衡量样本的稳定程度，比如样本在两个月份之间的变化是否稳定。通常变量的PSI值在0.1以下表示变化不太显著，在0.1到0.25之间表示有比较显著的变化，大于0.25表示变量变化比较剧烈，需要特殊关注。

接下来以`balance`为例子，确认两次抽样的样本分布是否接近。

> 根据业务需求，PSI分析也可以在数据分析或者特征预处理的时候进行。


In [50]:
stats_df = table_statistics(train_x['balance'])

In [51]:
min_val, max_val = stats_df['min'], stats_df['max']

In [52]:
from secretflow.stats import psi_eval
from secretflow.stats.core.utils import equal_range
import jax.numpy as jnp

split_points = equal_range(jnp.array([min_val, max_val]), 3)
balance_psi_score = psi_eval(train_x['balance'], test_x['balance'], split_points)

sf.reveal(balance_psi_score)

Array(0.000, dtype=float32)

#### 安全性讨论

PSI分析是一个单方运算，由数据owner的PYU Device执行计算。

### 逻辑回归模型

使用 **ml.linear.ss_sgd.SSRegression** 可以进行密态逻辑回归模型的训练。

请参考相关的API文档。


In [53]:
from secretflow.ml.linear.ss_sgd import SSRegression

lr_model = SSRegression(spu)
lr_model.fit(
    x=train_x,
    y=train_y,
    epochs=3,
    learning_rate=0.1,
    batch_size=1024,
    sig_type='t1',
    reg_type='logistic',
    penalty='l2',
    l2_norm=0.5,
)

INFO:root:epoch 1 times: 2.9048001766204834s
INFO:root:epoch 2 times: 0.9002580642700195s
INFO:root:epoch 3 times: 0.8167884349822998s


你可能会对为何上面的语句很快就执行完毕感到困惑，原因是在隐语中，语句都是lazy evaluation的，在上面的例子中，直到lr_model被真正被使用的时候，**lr_model.fit**才会被执行。

#### 安全性讨论

SSRegression的训练基于SPU Device，双方的原始数据将会被保护。

### XGBoost模型

使用 **ml.boost.ss_xgb_v.Xgb** 可以进行密态XGBoost模型的训练。

请参考相关的API文档。

In [54]:
from secretflow.ml.boost.ss_xgb_v import Xgb

xgb = Xgb(spu)
params = {
    'num_boost_round': 3,
    'max_depth': 5,
    'sketch_eps': 0.25,
    'objective': 'logistic',
    'reg_lambda': 0.2,
    'subsample': 1,
    'colsample_by_tree': 1,
    'base_score': 0.5,
}
xgb_model = xgb.train(params=params, dtrain=train_x, label=train_y)

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorXgbTreeWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorXgbTreeWorker'> with party bob.
INFO:root:fragment_count 1
INFO:root:prepare time 0.18915605545043945s
INFO:root:global_setup time 2.070232391357422s
INFO:root:build & infeed bucket_map fragments [0, 0]
INFO:root:build & infeed bucket_map time 0.9691281318664551s
INFO:root:init_pred time 0.04111146926879883s
INFO:root:epoch 0 tree_setup time 0.11794877052307129s
INFO:root:fragment[0, 0] gradient sum time 1.0241131782531738s
INFO:root:level 0 time 1.3071274757385254s
INFO:root:fragment[0, 0] gradient sum time 1.461333990097046s
INFO:root:level 1 time 1.7634406089782715s
INFO:root:fragment[0, 0] gradient sum time 2.818866014480591s
INFO:root:level 2 time 3.1974093914031982s
INFO:root:fragment[0, 0] gradient sum time 5.653681039810181s
INFO:root:level 3 time 6.15212082862854s
INFO:root:fragment[0, 0] gradient sum time 11.2245

Xgb.train将会直接执行，请耐心等待。

#### 安全性讨论

Xgb的训练基于SPU Device，双方的原始数据将会被保护。

## 模型预测

接下来，我们将会分别利用刚刚训练好的模型来预测测试集。

### 逻辑回归模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [56]:
lr_y_hat = lr_model.predict(x=test_x, batch_size=1024, to_pyu=bob)

In [64]:
sf.reveal(lr_y_hat.partitions[bob])

array([[0.103],
       [0.254],
       [0.078],
       ...,
       [0.107],
       [0.161],
       [0.509]], dtype=float32)

In [68]:
sf.reveal(test_x.partitions[bob].data)

,duration,campaign,pdays,previous,contact_cellular,contact_telephone,contact_unknown,month_1.0,month_2.0,month_3.0,...,day_26.0,day_27.0,day_28.0,day_29.0,day_30.0,day_31.0,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,-0.027777,0.393473,-0.411165,-0.243411,0.734665,-0.261265,-0.633888,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951
1,1.891149,-0.246119,-0.411165,-0.243411,-1.361165,-0.261265,1.577565,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951
2,0.395860,-0.565915,-0.411165,-0.243411,-1.361165,-0.261265,1.577565,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951
3,1.522131,-0.565915,-0.411165,-0.243411,0.734665,-0.261265,-0.633888,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951
4,0.794806,-0.246119,1.246629,1.456101,0.734665,-0.261265,-0.633888,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,2.867827,-0.20515,-0.185798,-2.118863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7320,-0.442209,-0.565915,-0.411165,-0.243411,0.734665,-0.261265,-0.633888,-0.17819,4.010074,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951
7321,-0.027777,-0.246119,-0.411165,-0.243411,0.734665,-0.261265,-0.633888,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951
7322,0.096951,-0.246119,-0.411165,-0.243411,0.734665,-0.261265,-0.633888,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951
7323,0.096951,-0.565915,-0.411165,-0.243411,0.734665,-0.261265,-0.633888,-0.17819,-0.249372,-0.102398,...,-0.152283,-0.158368,-0.206231,-0.201435,-0.18861,-0.12153,-0.348696,-0.20515,-0.185798,0.471951


#### 安全性讨论

逻辑回归的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

### XGBoost模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [69]:
xgb_y_hat = xgb_model.predict(dtrain=test_x, to_pyu=bob)

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorXgbTreeWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorXgbTreeWorker'> with party bob.


#### 安全性讨论

XGBoost模型的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

## 模型评估

接下来，我们将利用测试数据集对模型效果进行评估，包括：

- 二分类评估
- PVA
- P-Value
- 评分卡转换

### 二分类评估

隐语中对二分类的评估有集成的支持。

`BiClassificationEval` 将计算 `AUC`, `KS`, `F1 Score`, `Lift`, `K-S`, `Gain`, `Precision`, `Recall` 等统计数值， 并提供（基于prediction score的）等频和等距分箱的统计报告和总报告。

不同分桶中评估模型的预测的`threshold`不同。总报告中依赖`threshold`的统计取的是各个分桶的最佳值。

详情可以参考API文档。

In [70]:
from secretflow.stats.biclassification_eval import BiClassificationEval

biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=lr_y_hat, bucket_size=20
)
lr_report = sf.reveal(biclassification_evaluator.get_all_reports())

In [71]:
print(f'positive_samples: {lr_report.summary_report.positive_samples}')
print(f'negative_samples: {lr_report.summary_report.negative_samples}')
print(f'total_samples: {lr_report.summary_report.total_samples}')
print(f'auc: {lr_report.summary_report.auc}')
print(f'ks: {lr_report.summary_report.ks}')
print(f'f1_score: {lr_report.summary_report.f1_score}')

positive_samples: 845.0
negative_samples: 6480.0
total_samples: 7325.0
auc: 0.9002374410629272
ks: 0.6575827598571777
f1_score: nan


In [72]:
biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=xgb_y_hat, bucket_size=20
)
xgb_report = sf.reveal(biclassification_evaluator.get_all_reports())

In [73]:
print(f'positive_samples: {xgb_report.summary_report.positive_samples}')
print(f'negative_samples: {xgb_report.summary_report.negative_samples}')
print(f'total_samples: {xgb_report.summary_report.total_samples}')
print(f'auc: {xgb_report.summary_report.auc}')
print(f'ks: {xgb_report.summary_report.ks}')
print(f'f1_score: {xgb_report.summary_report.f1_score}')

positive_samples: 845.0
negative_samples: 6480.0
total_samples: 7325.0
auc: 0.8731328248977661
ks: 0.6151344180107117
f1_score: 0.5083114504814148


### 预测偏差

结果由`abs(mean(Acutal) - mean(Prediction))`计算获得, 值越小越好。

In [74]:
from secretflow.stats import prediction_bias_eval

prediction_bias = prediction_bias_eval(
    test_y, lr_y_hat, bucket_num=4, absolute=True, bucket_method='equal_width'
)

sf.reveal(prediction_bias)

PredictionBiasReport(buckets=[BucketPredictionBiasReport(left_endpoint=0.0, left_closed=True, right_endpoint=0.25, right_closed=False, isna=False, avg_prediction=0.0, avg_label=0.13856597244739532, bias=0.13856597244739532, absolute=True), BucketPredictionBiasReport(left_endpoint=0.25, left_closed=True, right_endpoint=0.5, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.5, left_closed=True, right_endpoint=0.75, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.75, left_closed=True, right_endpoint=1.0, right_closed=True, isna=False, avg_prediction=1.0, avg_label=0.362821102142334, bias=0.637178897857666, absolute=True)])

In [75]:
xgb_pva_score = prediction_bias_eval(
    test_y, xgb_y_hat, bucket_num=4, absolute=True, bucket_method='equal_width'
)

sf.reveal(xgb_pva_score)

PredictionBiasReport(buckets=[BucketPredictionBiasReport(left_endpoint=0.0, left_closed=True, right_endpoint=0.25, right_closed=False, isna=False, avg_prediction=0.0, avg_label=0.15805619955062866, bias=0.15805619955062866, absolute=True), BucketPredictionBiasReport(left_endpoint=0.25, left_closed=True, right_endpoint=0.5, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.5, left_closed=True, right_endpoint=0.75, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.75, left_closed=True, right_endpoint=1.0, right_closed=True, isna=False, avg_prediction=1.0, avg_label=0.3796447515487671, bias=0.6203552484512329, absolute=True)])

### P-Value
双方可通过p-value的值来判断参数是否显著，即该自变量是否可以有效预测因变量的变异, 从而判定对应的解释变量是否应包括在模型中。


In [76]:
from secretflow.stats import SSPValue

model = lr_model.save_model()
sspv = SSPValue(spu)
pvalues = sspv.pvalues(test_x, test_y, model)

pvalues

AttributeError: 'PValue' object has no attribute 'pvalues'

### 评分卡转换

> 严格来说，评分卡转化是对预测结果的后续处理，并不属于模型评估。


我们将 `y = 1` 的概率设为`p`， `odds = p / (1 - p)`, 评分卡设定的分值刻度可以通过将分值表示为比率对数的线性表达式来定义，即可表示为下式：

`Score = A - B log(odds)`， A 和 B 是可以设定的常数。隐语中提供了评分卡转换功能，详情可以参考API文档。

In [77]:
from secretflow.stats import BiClassificationEval, ScoreCard

sc = ScoreCard(20, 600, 20)
score = sc.transform(xgb_y_hat)

sf.reveal(score.partitions[bob])

array([[452.600],
       [503.873],
       [452.600],
       ...,
       [452.600],
       [454.726],
       [488.490]])

### 安全性讨论

以上所有模型评估的方法均为单方运算，由label拥有者的PYU Device进行运算。

## 实验结束

最后，我们需要清理临时文件，并关闭隐语cluster。

In [78]:
import os

try:
    os.remove(alice_path)
    os.remove(alice_psi_path)
    os.remove(bob_path)
    os.remove(bob_psi_path)
except OSError:
    pass

sf.shutdown()

恭喜！你已经完成了隐语金融风控全链路的全部实验内容。

如果你对本实验有任何建议和问题，请在[Github Issues](https://github.com/secretflow/secretflow/issues)上联系我们。